# Cluster Forecasting

In [205]:
import pandas as pd
import sketch
import ast

import lightgbm as lgb 
from sklearn.metrics import mean_absolute_error, r2_score

In [268]:
clients_per_clusters_path = 'clean_data/clients_per_clusters.csv'
clients_and_clusters_path = 'clean_data/clients_and_clusters.csv'
transactions_path = 'train_data_npo/npo_trnsctns.csv'
contributors_path = 'train_data_npo/npo_cntrbtrs.csv'

clients_features = {
    "clnt_id" : "ID клиента",
    "slctn_nmbr" : "Номер выборки",
    "gndr" : "Пол клиента: м: 0, ж: 1",
    "age" : "Возраст клиента, лет",
    "brth_yr" : "Год рождения клиента",
    "pstl_code" : "Почтовый индекс",
    "city" : "Живет клиент в городе (1) или нет (0)"
}

contributors_features = {
    "npo_accnt_id" : "ID счета клиента",
    "client_id" : "ID клиента",
    "slctn_nmbr" : "Номер выборки",
    "accnt_pnsn_schm" : "Условный код пенсионной схемы счета клиента",
    "npo_accnt_status" : "Статус счета клиента: Закрыт: 0, Открыт: 1",
    "npo_accnt_status_date" : "Дата статуса счета клиента",
    "npo_blnc" : "Баланс счета клиента, руб.",
    "npo_pmnts_sum" : "Сумма взносов клиента, руб.",
    "npo_pmnts_nmbr" : "Число взносов клиента",
    "npo_frst_pmnt_date" : "Дата первого взноса клиента",
    "npo_lst_pmnt_date" : "Дата последнего взноса клиента",
    "npo_ttl_incm" : "Сумма дохода, начисленного на счет клиента, руб."
}

transactions_features = {
    "npo_accnt_id" : "ID счета клиента",
    "slctn_nmbr" : "Номер выборки",
    "npo_sum" : "Размер операции по счету клиента, руб.",
    "npo_operation_date" : "Дата взноса клиента",
    "npo_operation_group" : "Тип операции по счету клиента: Поступление взносов: 0, Начисление дохода: 1",
}

In [269]:
contributors = pd.read_csv('train_data_npo/npo_cntrbtrs.csv')
contributors.head(3)

,npo_accnt_id,clnt_id,accnt_pnsn_schm,slctn_nmbr,npo_accnt_status,npo_accnt_status_date,npo_blnc,npo_pmnts_sum,npo_pmnts_nmbr,npo_frst_pmnt_date,npo_lst_pmnt_date,npo_ttl_incm
0,0x90B7458B8CBFF24980DEC312BA4A1AF5,0x85390230E8955E4FA736E62B0F0E3844,1.0,0,1,2001-05-14,10158.96,2276.42,1.0,2005-08-31,2005-08-31,5638.83
1,0xC64D3161D31A8441A65224792D370CB3,0xC2B51FD4FE57F7479210FD7258DF5B0B,3.0,2,0,2018-10-30,NaN,230084.40,55.0,2013-03-07,2017-09-10,39875.30
2,0xC92F1AA5587E2348BEF17432FBD6C2E6,0x8EC850934FF06A4AA0A856CF43B8D666,4.0,1,0,2014-01-20,NaN,7921.95,8.0,2012-11-26,2013-06-30,207.50


In [270]:
clients_per_clusters = pd.read_csv(clients_per_clusters_path, index_col=0)
clients_per_clusters.head(3)

,cluster,clnt_id
0,0,"['0x896FDAA89D08B44698884BC6876C3455', '0x7F16..."
1,1,"['0x88C8CB57D2D6B14393894C0CBB8A9A4A', '0x1441..."
2,2,"['0xE161325D909F9848979ABFE4137216CE', '0xE459..."


In [271]:
clients_and_clusters = pd.read_csv(clients_and_clusters_path, index_col=0)
clients_and_clusters.head(3)

,clnt_id,cluster,age,city,gndr
0,0xD1930AC934CD0D4AB6141DF45637EFE4,3,74.0,0,1.0
1,0x25DCE99C94913C42A49F739DDA3AE81A,3,62.0,0,0.0
2,0xCF29021EFE24454693866565B7CAB0D8,3,69.0,0,1.0


In [272]:
transactions = pd.read_csv(transactions_path)
transactions.head(3)

,npo_accnt_id,npo_sum,slctn_nmbr,npo_operation_date,npo_operation_group
0,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2011-11-11 00:00:00,0
1,0x05C7DF8BA2611640BE946E29CF20C6D2,8394.05,0,2010-08-30 00:00:00,1
2,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2007-03-12 00:00:00,0


In [273]:
client_payments = transactions.loc[transactions['npo_operation_group'] == 0, 
    ['npo_accnt_id', 'npo_sum', 'slctn_nmbr', 'npo_operation_date']]
# Rename column
#client_payments.rename(columns={'npo_accnt_id':'clnt_id'}, inplace=True)
#client_payments['clnt_id'] = client_payments['clnt_id'].astype(str)
client_payments.head(3)

,npo_accnt_id,npo_sum,slctn_nmbr,npo_operation_date
0,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2011-11-11 00:00:00
2,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2007-03-12 00:00:00
4,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2016-11-05 00:00:00


In [274]:
client_payments = pd.merge(client_payments, contributors.loc[:,['clnt_id', 'npo_accnt_id']], on='npo_accnt_id')
client_payments.head(3)

,npo_accnt_id,npo_sum,slctn_nmbr,npo_operation_date,clnt_id
0,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2011-11-11 00:00:00,0xE1494CDD114361469D1979AB0311F504
1,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2007-03-12 00:00:00,0xE1494CDD114361469D1979AB0311F504
2,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2016-11-05 00:00:00,0xE1494CDD114361469D1979AB0311F504


In [275]:
client_payments = pd.merge(client_payments, clients_and_clusters, on='clnt_id')
client_payments.head(3)

,npo_accnt_id,npo_sum,slctn_nmbr,npo_operation_date,clnt_id,cluster,age,city,gndr
0,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2011-11-11 00:00:00,0xE1494CDD114361469D1979AB0311F504,3,59.0,0,0.0
1,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2007-03-12 00:00:00,0xE1494CDD114361469D1979AB0311F504,3,59.0,0,0.0
2,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2016-11-05 00:00:00,0xE1494CDD114361469D1979AB0311F504,3,59.0,0,0.0


In [277]:
# Format npo_operation_date to quarter
client_payments['quarter'] = pd.to_datetime(client_payments['npo_operation_date']).dt.to_period('Q')

In [280]:
quarterly_data = client_payments.groupby(['quarter', 'cluster']).agg({'npo_sum': 'sum', 'age': 'mean', 'clnt_id': 'count'})
quarterly_data.reset_index(inplace=True)
quarterly_data.tail()
quarterly_data = quarterly_data.loc[quarterly_data['quarter'] >= pd.Period('2005Q1')].reset_index(drop=True)
quarterly_data.head(10)

,quarter,cluster,npo_sum,age,clnt_id
0,2005Q1,0,613551.01,64.488104,1387
1,2005Q1,1,107699.44,58.848131,428
2,2005Q1,2,477613.70,65.320218,1652
3,2005Q1,3,19635507.11,58.369340,42733
4,2005Q2,0,705694.74,63.709695,1805
5,2005Q2,1,116742.89,58.639080,435
6,2005Q2,2,550028.02,65.098351,1637
7,2005Q2,3,36552837.30,58.175508,43622
8,2005Q3,0,12951672.15,62.183311,6028
9,2005Q3,1,8195453.94,63.691004,1534


In [281]:
quarterly_data['quarter'] = quarterly_data['quarter'].dt.to_timestamp()
quarterly_data['quarter']

0     2005-01-01
1     2005-01-01
2     2005-01-01
3     2005-01-01
4     2005-04-01
         ...    
275   2022-01-01
276   2022-04-01
277   2022-04-01
278   2022-04-01
279   2022-04-01
Name: quarter, Length: 280, dtype: datetime64[ns]

In [282]:
df_moex = pd.read_csv('df_moex', index_col=0)
df_moex['quarter'] = pd.to_datetime(df_moex['quarter'])
df_rgb = pd.read_csv('df_rgb', index_col=0)
df_rgb['quarter'] = pd.to_datetime(df_rgb['quarter'])
df_employ = pd.read_csv('df_employ', index_col=0)
df_rgb.head(3)

/var/folders/h2/nrzj558d5nbgkp248n_sv6gm0000gn/T/ipykernel_47304/4075141266.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_moex['quarter'] = pd.to_datetime(df_moex['quarter'])
/var/folders/h2/nrzj558d5nbgkp248n_sv6gm0000gn/T/ipykernel_47304/4075141266.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_rgb['quarter'] = pd.to_datetime(df_rgb['quarter'])


,quarter,Цена,Откр.,Макс.,Мин.,Объём,Изм. %
0,2005-01-01,11362,"114,24","114,35","113,23",NaN,"-0,64%"
1,2005-04-01,11288,"112,90","113,16","112,80",NaN,"0,00%"
2,2005-07-01,11283,"112,85","112,99","112,54",NaN,"-0,01%"


In [283]:
data_quarters = []
for _, row in df_employ.iterrows():
        year = row['year'][:4]
        bond_price = row['value']

        # Generate data for each of the four quarters of the year
        for quarter in range(1, 5):
            data_quarters.append({'quarter': f'{year}Q{quarter}', 'employ': bond_price})

    # Create the new DataFrame with quarterly values
df_employ = pd.DataFrame(data_quarters)
df_employ['quarter'] = pd.to_datetime(df_employ['quarter'])

/var/folders/h2/nrzj558d5nbgkp248n_sv6gm0000gn/T/ipykernel_47304/3180973414.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_employ['quarter'] = pd.to_datetime(df_employ['quarter'])


In [284]:
# Join quarterly_data with df_moex on quarter
quarterly_data = pd.merge(quarterly_data, df_moex, on='quarter')
quarterly_data.rename(columns={'VALUE': 'moex'}, inplace=True)

In [285]:
# Join quarterly_data with df_moex on quarter
quarterly_data = pd.merge(quarterly_data, df_rgb.loc[:,['quarter', 'Цена']], on='quarter')

quarterly_data.rename(columns={'Цена': 'rgb_price'}, inplace=True)

#quarterly_data.drop(columns=['Объём'], inplace=True)

In [286]:
quarterly_data = pd.merge(quarterly_data, df_employ, on='quarter')
quarterly_data.head(3)

,quarter,cluster,npo_sum,age,clnt_id,moex,rgb_price,employ
0,2005-01-01,0,613551.01,64.488104,1387,2.452452e+10,11362,5666.0
1,2005-01-01,1,107699.44,58.848131,428,2.452452e+10,11362,5666.0
2,2005-01-01,2,477613.70,65.320218,1652,2.452452e+10,11362,5666.0


In [287]:
quarterly_data = quarterly_data.set_index('quarter')
quarterly_data['npo_sum'] = quarterly_data['npo_sum'].astype('float')
quarterly_data['moex'] = quarterly_data['moex'].astype('float')
quarterly_data['rgb_price'] = quarterly_data['rgb_price'].astype('float')
quarterly_data['employ'] = quarterly_data['employ'].astype('float')

In [288]:
quarterly_data.to_csv('clean_data/quarterly_data.csv')

In [290]:
qdata_0 = quarterly_data.loc[quarterly_data.cluster == 0, quarterly_data.columns != 'cluster']
qdata_1 = quarterly_data.loc[quarterly_data.cluster == 1, quarterly_data.columns != 'cluster']
qdata_2 = quarterly_data.loc[quarterly_data.cluster == 2, quarterly_data.columns != 'cluster']
qdata_3 = quarterly_data.loc[quarterly_data.cluster == 3, quarterly_data.columns != 'cluster']
qdata_0.head(3)

,npo_sum,age,clnt_id,moex,rgb_price,employ
quarter,,,,,,
2005-01-01,613551.01,64.488104,1387,2.452452e+10,11362.0,5666.0
2005-04-01,705694.74,63.709695,1805,1.532838e+10,11288.0,5666.0
2005-07-01,12951672.15,62.183311,6028,1.172042e+10,11283.0,5666.0


In [291]:
def train_test_split(df):
    train_size = int(len(df) * 0.8)
    train_data, test_data = df.iloc[:train_size, :], df.iloc[train_size:, :]
    X_train = train_data.loc[:, df.columns != 'npo_sum']
    y_train = train_data.loc[:, 'npo_sum']
    X_test = test_data.loc[:, df.columns != 'npo_sum']
    y_test = test_data.loc[:, 'npo_sum']
    return X_train, y_train, X_test, y_test

## Forecasting using LightGBM

In [292]:
def train(df):
    X_train, y_train, X_test, y_test = train_test_split(df)
    regressor = lgb.LGBMRegressor()
    regressor.fit(X_train, y_train)
    pred = regressor.predict(X_test)
    return (mean_absolute_error(y_test, pred), r2_score(y_test, pred))

In [296]:
mae, r2 = train(qdata_0)
print(mae, r2)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001396 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 95
[LightGBM] [Info] Number of data points in the train set: 55, number of used features: 5
[LightGBM] [Info] Start training from score 21670939.968182
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

По дате предсказываем:
1. MOEX
2. RGB
3. EMPLOY

По ним предсказываем NPO_SUM